In [1]:
!pip install -q transformers accelerate datasets torchaudio librosa torchcodec --quiet

import torch
import torchaudio
from transformers import pipeline

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "MohamedRashad/Arabic-Whisper-CodeSwitching-Edition"

pipe = pipeline(
    "automatic-speech-recognition",
    model=model_id,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda


In [24]:
def prepare_audio(audio_path):
    waveform, sr = torchaudio.load(audio_path)

    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        waveform = resampler(waveform)

    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    return waveform.squeeze().numpy()

audio_input = prepare_audio("noise_arabic_cleaned.wav")

In [25]:
tokenizer = pipe.tokenizer

def transcribe_code_switched(audio_path):
    audio_array = prepare_audio(audio_path)

    # prompt = "Transcribe ال audio ده ل عربي مصري بس ركز انك لازم تحافظ على الكلام ال English زي ما هو"
    # prompt = "Transcribe this audio into English ONLY"
    prompt = "طلع النص بتاع الصوت ده بالعربي المصري بس و بلاش اي لغة تانية "
    prompt_ids = tokenizer.get_prompt_ids(prompt, return_tensors="pt").to(device)

    result = pipe(
        audio_array,
        return_timestamps=True,
        generate_kwargs={
            "temperature": 0,
            "language": "arabic",
            "task": "transcribe",
            "num_beams": 5,
            "prompt_ids": prompt_ids
        }
    )
    return result

result = transcribe_code_switched("noise_arabic_cleaned.wav")

print(result["text"])

زيادت الصوت واضح و لا فيه دوشة لأ فيه شوية دوشة بس ده المطلوب عشان نشوف النظام هيدر يفهم الكلام و لا لأ صح لما جربته مرة لفترة كان بيقدر يرقط الكلام بس لو الدوشة زادت بيبدأ يغلط يعنى طيب نجرب دلوقت أنا هفضل أتكلم و أنت شغل المروح تمام و كده نشوف بقى هل الكلام هيفضل باين في النص و لا الدوشة تغطي عليه صوت المروح عالي جدا سمع؟أيوة سامع بس مافيش مشكلة احنا عايزين كده طيب قول اي جملة عادية كده زي الجلسة الجاية هنتكلم عن النتائج اللي طلعت الجلسة الجاية هنتكلم عن النتائج اللي طلعت و هنحاول نحسن الدقة في التسجيل كويس، دلوقت خليك بعيد عن الميكروفون و اتكلم تاني طيب الصوت كده واضح ولا واطي؟ لأ كده حلو جدا، عايزين نشوف بقى هل النظام هيقدر يفرق من الدوشة و الكلام؟ان شاء الله لو نجح فى التسجيل ده يبقى نموذج قوي جدا فى التعامل مع الدوش تمام يللا نسجل دلوقت و نشوف فى النتيجة بعدين زي الفور شكرا يا زيكى عفوا
